In [ ]:
!git clone https://github.com/DrikDrik/sber-movqgan-project

In [ ]:
import sys
sys.path.append('/content/sber-movqgan-project/')

In [ ]:
!pip install pip==24.0
!pip install -r 'sber-movqgan-project/requirements.txt'

In [ ]:
from models.GAN.model import generate_new_codes, Critic, Generator
from models.GAN.train import training_loop
from pipeline.movqgan import decode
from pipeline.show import show_images

In [ ]:
from utils.imports import *

In [ ]:
from pipeline.movqgan import get_model, decode
model = get_model()

In [ ]:
from utils import download_datasets
download_datasets.main()

In [ ]:
!unzip -q datasets/latent_dataset.zip -d '/content/latent_representations/'

In [ ]:
from pipeline.get_latent import LatentDataset

In [ ]:
import os
temp_dataset = LatentDataset('latent_representations')
temp_loader = DataLoader(temp_dataset, batch_size=128, shuffle=True)

In [ ]:
from utils import normalize
normalize_fn = normalize.normalize_fn(temp_loader)
inverse_normalize_fn = normalize.inverse_normalize_fn(temp_loader)

In [ ]:
transform = transforms.Compose([
    transforms.Lambda(normalize_fn)
])
dataset = LatentDataset('latent_representations', transform=transform)
loader = DataLoader(dataset, batch_size=200, shuffle=True)

In [ ]:
store_path = "/content/checkpoints/gan1.ckpt"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = Generator(latent_dim=128).to(device)
critic = Critic().to(device)
optim_g = optim.Adam(generator.parameters(), lr=0.0001, betas=(0.5, 0.9))
optim_c = optim.Adam(critic.parameters(), lr=0.0001, betas=(0.5, 0.9))
if not os.path.isdir(os.path.dirname(store_path)): os.mkdir(os.path.dirname(store_path))


In [ ]:
training_loop(generator, critic, loader, model=model, n_epochs=100, optim_g=optim_g, optim_c=optim_c, device=device, n_sched=7, n_critic=10, inverse_normalize_fn=inverse_normalize_fn, store_path=store_path)